In [ ]:
with open("C_example.txt", "r") as file:
	C_example = file.read()
with open("P_example.txt", "r") as file:
  P_example = file.read()
def init_template(type, example):
    return {
    "custom_id": None, 
    "method": "POST", 
    "url": "/v1/chat/completions",
    "body": {"model": "gpt-4.1", 
             "messages": 
                 [{"role": "system", "content": f"""
                   You are an Ontology engineer. 
                   Generate a {type} description including information of axioms and current description. 
                   The description should be concise and informative, providing a clear understanding of the {type}'s purpose and characteristics.
                   Don't generate unnecessary text. Just generate {type} description only."""},
                   {"role": "user", "content": f"""
                   Generate a {type} description including information of axioms and current description. 
                   If there is no description, generate a description based on the axiom."""},
                   {"role": "user", "content": "For example, "+example},
                   {"role": "user", "content": "Now, generate the description using class axiom and current description below."}
                  ],
             "max_tokens": 1024 
             }
    }

In [ ]:
import os
directory_path = "Ontology_description"
ontology_list = {file.split("_")[0]: {"description":os.path.join(directory_path, file)} for file in os.listdir(directory_path)}
for file in os.listdir("Axiom_per_entity"):
    ontology_list[file.split("_")[0]]["axiom"]=os.path.join("Axiom_per_entity", file)

{'event': {'description': 'Ontology_description/event_description.json', 'axiom': 'Axiom_per_entity/event_axiom.json'}, 'stuff': {'description': 'Ontology_description/stuff_description.json', 'axiom': 'Axiom_per_entity/stuff_axiom.json'}, 'OntoDT': {'description': 'Ontology_description/OntoDT_description.json', 'axiom': 'Axiom_per_entity/OntoDT_axiom.json'}, 'time': {'description': 'Ontology_description/time_description.json', 'axiom': 'Axiom_per_entity/time_axiom.json'}, 'AfricanWildlifeOntology1': {'description': 'Ontology_description/AfricanWildlifeOntology1_description.json', 'axiom': 'Axiom_per_entity/AfricanWildlifeOntology1_axiom.json'}, 'pizza': {'description': 'Ontology_description/pizza_description.json', 'axiom': 'Axiom_per_entity/pizza_axiom.json'}, 'swo': {'description': 'Ontology_description/swo_merged_description.json', 'axiom': 'Axiom_per_entity/swo_merged_axiom.json'}, 'exchangemodel': {'description': 'Ontology_description/exchangemodel_description.json', 'axiom': 'Axi

In [ ]:
import json
from copy import deepcopy
import random
data_num = 0
for ontology in ontology_list:
    batches = []
    cls_num, pro_num = 0, 0
    with open(ontology_list[ontology]["description"], "r") as f:
        temp_desc=json.load(f)
    with open(ontology_list[ontology]["axiom"], "r") as f:
        temp_axiom=json.load(f)
    for cls in temp_axiom["classes"]:
        temp = deepcopy(init_template("class", C_example))
        temp["body"]["messages"].append({"role": "user", "content": "Class name: "+cls})
        temp["custom_id"] = ontology+"_"+cls
        temp["body"]["messages"].append({"role": "user", "content": "Axiom: "+str(temp_axiom["classes"][cls])})
        if cls in temp_desc["classes"]: temp["body"]["messages"].append({"role": "user", "content": "Current description: "+str(temp_desc["classes"][cls])})
        temp["body"]["messages"].append({"role": "user", "content": "Generated description: "})
        batches.append(temp)
    cls_num=len(batches)
    print(f"Ontology name: {ontology}, {cls_num} classes")
    for prop in temp_axiom["properties"]:
        temp = deepcopy(init_template("property", P_example))
        temp["body"]["messages"].append({"role": "user", "content": "Property name: "+prop})
        temp["custom_id"] = ontology+"_"+prop
        temp["body"]["messages"].append({"role": "user", "content": "Axiom: "+str(temp_axiom["properties"][prop])})
        if prop in temp_desc["properties"]: temp["body"]["messages"].append({"role": "user", "content": "Current description: "+str(temp_desc["properties"][prop])})
        temp["body"]["messages"].append({"role": "user", "content": "Generated description: "})
        pro_num+=1
        batches.append(temp)
    
    pro_num=len(batches)-cls_num
    print(f"Ontology name: {ontology}, {pro_num} properties")
    # Save the batches to a JSONL file
    with open("Batchinput/"+ontology+'_batchinput.jsonl', 'w', encoding='utf-8') as file:
        for item in batches:
            json_string = json.dumps(item, ensure_ascii=False)
            file.write(json_string + '\n')
    print(f"Ontology name: {ontology}, Number of batches: {len(batches)}")
    data_num += len(batches)
print(f"Total number of batches: {data_num}")
    
    

Ontology name: event, 122 classes
Ontology name: event, 14 properties
Ontology name: event, Number of batches: 136
Ontology name: stuff, 61 classes
Ontology name: stuff, 33 properties
Ontology name: stuff, Number of batches: 94
Ontology name: OntoDT, 402 classes
Ontology name: OntoDT, 17 properties
Ontology name: OntoDT, Number of batches: 419
Ontology name: time, 11 classes
Ontology name: time, 41 properties
Ontology name: time, Number of batches: 52
Ontology name: AfricanWildlifeOntology1, 27 classes
Ontology name: AfricanWildlifeOntology1, 5 properties
Ontology name: AfricanWildlifeOntology1, Number of batches: 32
Ontology name: pizza, 99 classes
Ontology name: pizza, 8 properties
Ontology name: pizza, Number of batches: 107
Ontology name: swo, 490 classes
Ontology name: swo, 10 properties
Ontology name: swo, Number of batches: 500
Ontology name: exchangemodel, 16 classes
Ontology name: exchangemodel, 12 properties
Ontology name: exchangemodel, Number of batches: 28
Ontology name: h

In [ ]:
from openai import OpenAI
import time
client = OpenAI(api_key="")

for ontology in os.listdir("Batchinput"):
  if ontology.endswith("stuff_batchinput.jsonl"):
    with open(os.path.join("Batchinput", ontology), "rb") as file:
      batch_input_file = client.files.create(
        file=file, 
        purpose="batch"
      )
    batch_input_file_id = batch_input_file.id

# Create a batch job
    client.batches.create(
      input_file_id=batch_input_file_id,
      endpoint="/v1/chat/completions",
      completion_window="24h", 
            )
  else:
    print(f"Skipping non-JSONL file: {ontology}")


Skipping non-JSONL file: OntoDT_batchinput.jsonl
Skipping non-JSONL file: event_batchinput.jsonl
Skipping non-JSONL file: swo_batchinput.jsonl
Skipping non-JSONL file: home_batchinput.jsonl
Skipping non-JSONL file: lab_batchinput.jsonl
Skipping non-JSONL file: AfricanWildlifeOntology1_batchinput.jsonl
Skipping non-JSONL file: time_batchinput.jsonl
Skipping non-JSONL file: exchangemodel_batchinput.jsonl
Skipping non-JSONL file: pizza_batchinput.jsonl


In [37]:
import os
import json

batchoutput_directory = "Batchoutput"
jsonl_files = [file for file in os.listdir(batchoutput_directory) if file.endswith(".jsonl")]

for jsonl_file in jsonl_files:
    loaded_data = []
    file_path = os.path.join(batchoutput_directory, jsonl_file)
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            temp_data = json.loads(line)
            temp_out = {"class": temp_data["custom_id"].split("_",1)[1], "description": temp_data["response"]["body"]["choices"][0]["message"]["content"]}
            loaded_data.append(temp_out)
    # Save the loaded data to a new jsonl file
    output_file_path = os.path.join("generated description", f"{jsonl_file}")
    with open(output_file_path, "w", encoding="utf-8") as f:
        for item in loaded_data:
            json_string = json.dumps(item, ensure_ascii=False)
            f.write(json_string + '\n')
    print(f"Loaded {len(loaded_data)} records from {len(jsonl_files)} JSONL files.")

Loaded 94 records from 1 JSONL files.
